In [0]:
from IPython.display import Image, display

image_url = "https://cdn-blog.lawrencemcdaniel.com/wp-content/uploads/2021/01/30083957/aws-s3-logo.png"

display(Image(url=image_url, width=200))


**Databricks Notebook: Leitura de Arquivo CSV do S3**

In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/aws_key/,aws_key/,0,1734697757000
dbfs:/FileStore/tables/rootkey-1.csv,rootkey-1.csv,99,1734697745000


In [0]:
# Excluir o arquivo específico
#dbutils.fs.rm("dbfs:/FileStore/tables/aws_keys/", recurse=False)


In [0]:
%fs ls /FileStore/tables/rootkey-1.csv

path,name,size,modificationTime
dbfs:/FileStore/tables/rootkey-1.csv,rootkey-1.csv,99,1734697745000


In [0]:
import pandas as pd

# Caminho do arquivo CSV no Databricks
csv_path = "/dbfs/FileStore/tables/rootkey-1.csv"

credentials_df = pd.read_csv(csv_path)



In [0]:
# Desmontar o ponto de montagem antes de tentar montar novamente
#dbutils.fs.unmount("/mnt/s3_bucket")

**Montar o bucket S3 no Databricks**


In [0]:
# Extrair as credenciais
ACCESS_KEY = credentials_df.loc[0, 'Access key ID']
SECRET_KEY = credentials_df.loc[0, 'Secret access key']
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")

# Configurar o bucket S3
AWS_BUCKET_NAME = "raw-lve"
MOUNT_NAME = "/mnt/s3_bucket"

# Montar o bucket S3 no Databricks
dbutils.fs.mount(
  source=f"s3a://{ACCESS_KEY}:{ENCODED_SECRET_KEY}@{AWS_BUCKET_NAME}",
  mount_point=MOUNT_NAME,
  extra_configs={"fs.s3a.access.key": ACCESS_KEY, "fs.s3a.secret.key": SECRET_KEY}
)


True

In [0]:
display(dbutils.fs.ls(MOUNT_NAME))

[FileInfo(path='dbfs:/mnt/s3_bucket/netflix_titles.csv', name='netflix_titles.csv', size=2410660, modificationTime=1734703938000)]

In [0]:
# Caminho do arquivo no S3
s3_path = f"{MOUNT_NAME}/netflix_titles.csv"

df_s3_movie = spark.read.option("header", "true").option("inferSchema", "true").csv(s3_path)

df_s3_movie.limit(5).display()


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole Howard, Jennifer Cameron, Jonathan Holmes, Lee Tockar, Lisa Durupt, Maya Kay, Michael Dobson","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies","Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first."
80117401,Movie,Jandino: Whatever it Takes,null,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,"""Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of """"Sex on Fire"""" in his comedy show."""
70234439,TV Show,Transformers Prime,null,"Peter Cullen, Sumalee Montano, Frank Welker, Jeffrey Combs, Kevin Michael Richardson, Tania Gunadi, Josh Keaton, Steve Blum, Andy Pessoa, Ernie Hudson, Daran Norris, Will Friedle",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron."
80058654,TV Show,Transformers: Robots in Disguise,null,"Will Friedle, Darren Criss, Constance Zimmer, Khary Payton, Mitchell Whitfield, Stuart Allan, Ted McGinley, Peter Cullen",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,"When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind."
80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins, Keith Powers, Alicia Sanz, Jake Borelli, Kid Ink, Yousef Erakat, Rebekah Graf, Anne Winters, Peter Gilroy, Patrick Davis",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,"When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity."


In [0]:
df_s3_movie.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [0]:

df_s3_movie.createOrReplaceTempView("netflix_titles_table")

In [0]:
%sql
SELECT * FROM netflix_titles_table LIMIT 5

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole Howard, Jennifer Cameron, Jonathan Holmes, Lee Tockar, Lisa Durupt, Maya Kay, Michael Dobson","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies","Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first."
80117401,Movie,Jandino: Whatever it Takes,null,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,"""Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of """"Sex on Fire"""" in his comedy show."""
70234439,TV Show,Transformers Prime,null,"Peter Cullen, Sumalee Montano, Frank Welker, Jeffrey Combs, Kevin Michael Richardson, Tania Gunadi, Josh Keaton, Steve Blum, Andy Pessoa, Ernie Hudson, Daran Norris, Will Friedle",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron."
80058654,TV Show,Transformers: Robots in Disguise,null,"Will Friedle, Darren Criss, Constance Zimmer, Khary Payton, Mitchell Whitfield, Stuart Allan, Ted McGinley, Peter Cullen",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,"When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind."
80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins, Keith Powers, Alicia Sanz, Jake Borelli, Kid Ink, Yousef Erakat, Rebekah Graf, Anne Winters, Peter Gilroy, Patrick Davis",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,"When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity."
